In [20]:
import os
import pandas as pd

def is_categorical_numeric(column, data, unique_threshold=10, frequency_threshold=0.05):
    """
    Determina se uma coluna numérica deve ser tratada como categórica.

    Args:
        column (str): Nome da coluna.
        data (pd.DataFrame): DataFrame contendo a coluna.
        unique_threshold (int): Número máximo de valores únicos para considerar como categórico.
        frequency_threshold (float): Frequência máxima de valores únicos em relação ao tamanho do dataset.

    Returns:
        bool: Verdadeiro se a coluna for categórica.
    """
    unique_values = data[column].nunique()
    if unique_values > unique_threshold:
        return False  # Muitos valores únicos, não é categórico
    value_frequency = unique_values / len(data)
    return value_frequency <= frequency_threshold  # Deve ter uma frequência pequena de valores únicos

def one_hot_encode_datasets(input_directory, output_directory):
    """
    Aplica one-hot encoding em todas as bases de dados de um diretório que possuem colunas categóricas,
    com tratamento de valores faltantes.

    Args:
        input_directory (str): Diretório contendo os arquivos .txt de entrada.
        output_directory (str): Diretório onde os arquivos transformados serão salvos.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in os.listdir(input_directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(input_directory, filename)
            try:
                # Carregar o dataset
                data = pd.read_csv(file_path, delimiter=',')

                # Contar valores faltantes
                missing_values_count = data.isnull().sum().sum()
                print(f"Arquivo '{filename}' contém {missing_values_count} valores faltantes.")

                # Identificar colunas categóricas do tipo objeto
                categorical_columns = list(data.select_dtypes(include=['object']).columns)

                # Identificar colunas categóricas numéricas com base em critérios mais robustos
                numeric_categorical_columns = [
                    col for col in data.select_dtypes(include=['number']).columns
                    if is_categorical_numeric(col, data) and col != data.columns[-1]  # Ignorar a coluna alvo
                ]

                # Combinar ambas as categorias identificadas
                all_categorical_columns = categorical_columns + numeric_categorical_columns

                # Tratar valores faltantes
                # Preencher valores faltantes para colunas categóricas com a moda
                for col in categorical_columns:
                    if col in data.columns and data[col].isnull().sum() > 0:
                        mode_value = data[col].mode()[0]
                        missing_count = data[col].isnull().sum()
                        print(f"Coluna categórica '{col}': {missing_count} valores faltantes preenchidos com a moda '{mode_value}'.")
                        data[col].fillna(mode_value, inplace=True)
                
                # Preencher valores faltantes para colunas numéricas com a média
                for col in data.select_dtypes(include=['number']).columns:
                    if col in data.columns and data[col].isnull().sum() > 0:
                        mean_value = data[col].mean()
                        missing_count = data[col].isnull().sum()
                        print(f"Coluna numérica '{col}': {missing_count} valores faltantes preenchidos com a média '{mean_value:.2f}'.")
                        data[col].fillna(mean_value, inplace=True)

                if len(all_categorical_columns) > 0:
                    print(f"Aplicando one-hot encoding no arquivo: {filename}")

                    # Aplicar one-hot encoding
                    data_encoded = pd.get_dummies(data, columns=all_categorical_columns, drop_first=False)

                    # Garantir que a coluna alvo seja a última
                    target_column = data.columns[-1]
                    columns = [col for col in data_encoded.columns if col != target_column] + [target_column]
                    data_encoded = data_encoded[columns]

                    # Salvar o dataset transformado
                    output_file_path = os.path.join(output_directory, f"encoded_{filename}")
                    data_encoded.to_csv(output_file_path, index=False, sep=',')

                    print(f"Arquivo transformado salvo em: {output_file_path}")
                else:
                    print(f"Não foi encontrada nenhuma coluna categórica no arquivo: {filename}")

            except Exception as e:
                print(f"Erro ao processar o arquivo {filename}: {e}")

# Caminhos de entrada e saída
input_directory = "datasets"
output_directory = "datasets_encoded"

# Aplicar one-hot encoding nos datasets
one_hot_encode_datasets(input_directory, output_directory)


Arquivo 'china.txt' contém 0 valores faltantes.
Aplicando one-hot encoding no arquivo: china.txt
Arquivo transformado salvo em: datasets_encoded\encoded_china.txt
Arquivo 'cocomo81.txt' contém 0 valores faltantes.
Não foi encontrada nenhuma coluna categórica no arquivo: cocomo81.txt
Arquivo 'desharnais.txt' contém 0 valores faltantes.
Aplicando one-hot encoding no arquivo: desharnais.txt
Arquivo transformado salvo em: datasets_encoded\encoded_desharnais.txt
Arquivo 'maxwell.txt' contém 2 valores faltantes.
Coluna categórica 'dba': 2 valores faltantes preenchidos com a moda 'Relatnl'.
Aplicando one-hot encoding no arquivo: maxwell.txt
Arquivo transformado salvo em: datasets_encoded\encoded_maxwell.txt


C:\Users\CALEO\AppData\Local\Temp\ipykernel_1924\1004948224.py:65: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(mode_value, inplace=True)
